# Project: day 1
**Goal:**  Implement linear regression in python from scratch, and understand how sklearn has implemented linear regression. To do this:

1.   Implement linear regression with sklearn.
2.   Estimate coefficents and intercept without sklearn to find out how to do linear regresion without a library.
3.  Check that the results in 1. and 2. are roughly equal



**Metric**: Infinity norm of difference between implementation in 1. and 2.

In [339]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [340]:
predictors, response = load_boston(return_X_y=True)
index = int(len(predictors)*0.8)


#Split data 80/20 into train/test  
train_x = predictors[:index]
train_y = response[:index]
test_x = predictors[index:]
test_y = response[index:]

In [353]:
#Test for model
model = LinearRegression().fit(train_x, train_y)
y_pred = model.predict(test_x)

intercept_test = model.intercept_
coef_test = model.coef_
mse_test = mean_squared_error(test_y,y_pred)
r_sq_test = model.score(test_x,test_y)

#calculates find the elemen which maximizes |a_i-b_i|
def difference_infinity_norm(a,b):
  difference = np.absolute(a-b)
  max_element = max(difference)
  return(max_element)

In [342]:
#Check if coeficenst are given by
#The maximum likelihood estimator of the coffeficents
n,m = np.shape(train_x)
inverse = np.linalg.solve((train_x.T@train_x),np.identity(13))
B = inverse@train_x.T@train_y
print("Max norm of sklearn models prediction minus my own implementation with intercept:", difference_infinity_norm(B,coef_test))
#Error is quite large, so guessing the default parameters is using a design matrix
#help(model) show a parameter fit_intercept=bool, default = True
#check if setting this to False is equivalent to this approach:
model_no_intercept = LinearRegression(fit_intercept=False).fit(train_x, train_y)
coef_without_intercept = model_no_intercept.coef_
#Test fails despite both being almost identical
print("Max norm of sklearn models prediction minus my own implementation without intercept:", difference_infinity_norm(B,coef_without_intercept))
#Guessing this due to difference in precision
#Biggest error is realtively small at ~10^-{12}, but this confirms how 
#fit_intercept works!

Max norm of sklearn models prediction minus my own implementation with intercept: 13.215563883313086
Max norm of sklearn models prediction minus my own implementation without intercept: 3.955502592134508e-12


In [371]:
#Straightforward implementation is not the same due to not including the incetercept
#Add a column of ones to find intercept!
def create_design_matrix_lm(numpy_array):
  n,m = np.shape(numpy_array)
  column_of_ones = np.ones([n,1])
  design_matrix = np.c_[column_of_ones ,numpy_array]
  return(design_matrix)

design = create_design_matrix_lm(train_x)
n,m = np.shape(design)
inverse = np.linalg.solve((design.T@design),np.identity(m))
B = inverse@design.T@train_y
print("Max norm of sklearn models coefficents minus my implementation's coefficents with a design matrix:" ,difference_infinity_norm(B[1:],coef_test))
print("Max norm of difference between intercepts:", np.abs(B[0]-model.intercept_))


Max norm of sklearn models coefficents minus my implementation's coefficents with a design matrix: 1.7358559034619248e-11
Max norm of difference between intercepts: 3.453948238529847e-11


In [373]:
def fit(B,x):
  X = create_design_matrix_lm(x)
  return(np.dot(X,B))

def mse(y,y_pred):
  sum = 0
  for i in range(len(y)):
    sum += (y[i]-y_pred[i])**2
  return(sum/len(y))

def rss(y,y_pred):
  yhat = np.sum(y)/len(y)
  SStot = 0
  SSres = 0
  for i in range(len(y)):
    SSres += (y[i]-y_pred[i])**2
    SStot += (y[i]-yhat)**2
  return(1-(SSres/SStot))

y_pred = fit(B,test_x)
print("Norm of MSE's", mse_test-mse(test_y,y_pred))
print("Norm of R squared:" , np.abs(rss(test_y,y_pred)-r_sq_test))

Norm of MSE's -1.0587086762825493e-12
Norm of R squared: 3.977373985719623e-14


#Results: 
The max norm of the difference between sklearns predictions, and the predictions i got with my own implementation were smaller than $10^{-11}$